In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))
    if len(filenames) > 3:
        print("...")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
import csv
import cv2
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from copy import deepcopy
import tensorflow as tf
import tensorflow.keras.layers as tfl
# from __future__ import print_function, division
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms
import time
import copy

# cudnn.benchmark = True
# plt.ion()   # interactive mode
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [23]:
MODEL_PATH = 'model1.pt'
TRAIN_PATH = 'train.csv'
TEST_PATH = 'test.csv'

NUM_EPOCHS = 200
BATCH_SIZE = 16
device = torch.device("cuda:0")
# try device = "cuda" 
# and change your settings/accelerator to GPU if you want it to run faster

In [24]:
from torchvision.models import resnet18, ResNet18_Weights
# resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
# resnet18(weights="IMAGENET1K_V1")
# resnet18(pretrained=True)  # deprecated
# resnet18(True)  # deprecated

Task1

In [25]:
class TaskDataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = data

        # print(self.data)

        # self.data = make_dataset(TRAIN_PATH, alphabet, 36, 2)
        self.return_filename = return_filename
        self.root = root

    def __getitem__(self, index):
        data = self.data[index][:-1]
        target = self.data[index][-1]
        return torch.Tensor(data), torch.Tensor(np.array(target))

    def __len__(self):
        return len(self.data)

In [26]:
test_data = pd.read_csv(TEST_PATH)
train_data = pd.read_csv(TRAIN_PATH)


def clean(data):
    le = LabelEncoder()

    # Replace str by numbers
    data = deepcopy(data)
    cols = ["attribute_0"] + ["attribute_1"]
    for col in cols:
        data[col] = le.fit_transform(data[col])
        #print(le.classes_)

    data = data.drop(['id', 'product_code'], axis=1)

    # None values
    imputer = SimpleImputer(strategy='mean')
    final_data = pd.DataFrame(imputer.fit_transform(data))

    final_data.columns = data.columns

    return final_data




In [27]:
train = clean(train_data)
train = train.to_numpy()
train_ds = TaskDataset(train, root=TRAIN_PATH)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)

# for x, y in train_ds:
#     print(x,y)


# val_ds = Task1Dataset(val_data, root=TRAIN_PATH)
# val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, drop_last=False, shuffle=False)

# train.head(5)

In [28]:
class Model(nn.Module):
    def __init__(self, input_shape):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_shape, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [29]:
model = Model(input_shape=23)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [30]:
losses = []
accur = []
for i in tqdm(range(NUM_EPOCHS)):
    for j, (x_train, y_train) in enumerate(train_dl):
        # print(x_train)
        #calculate output
        output = model(x_train)

        #calculate loss
        loss = loss_fn(output, y_train.reshape(-1, 1))

        #accuracy
        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if i % 1 == 0:
        losses.append(loss)
        print("epoch {}\tloss : {}".format(i, loss))

  0%|          | 0/200 [00:00<?, ?it/s]

epoch 0	loss : 0.550836980342865
epoch 1	loss : 0.4156382381916046
epoch 2	loss : 0.26345425844192505
epoch 3	loss : 0.7034420967102051
epoch 4	loss : 0.4091135859489441
epoch 5	loss : 0.4046209454536438
epoch 6	loss : 0.5291353464126587
epoch 7	loss : 0.5894084572792053
epoch 8	loss : 0.6629515886306763
epoch 9	loss : 0.49991440773010254
epoch 10	loss : 0.5452114939689636
epoch 11	loss : 0.7098002433776855
epoch 12	loss : 0.652130663394928
epoch 13	loss : 0.5319000482559204
epoch 14	loss : 0.30324944853782654
epoch 15	loss : 0.5050954818725586
epoch 16	loss : 0.7304118275642395
epoch 17	loss : 0.36379125714302063
epoch 18	loss : 0.38245299458503723
epoch 19	loss : 0.7674342393875122
epoch 20	loss : 0.40940141677856445
epoch 21	loss : 0.7183772325515747
epoch 22	loss : 0.31352344155311584
epoch 23	loss : 0.6255540251731873
epoch 24	loss : 0.5630595088005066
epoch 25	loss : 0.6641683578491211
epoch 26	loss : 0.5449346899986267
epoch 27	loss : 0.4745880365371704
epoch 28	loss : 0.3264630

In [31]:
torch.save(model, MODEL_PATH)